In [1]:
import pysindy as ps

import deepSI
from deepSI import System_data
from deepSI.fit_systems import SS_encoder_general
from deepSI.fit_systems.encoders import default_encoder_net, default_state_net, default_output_net
from deepSI.fit_systems.fit_system import My_Simple_DataLoader, print_array_byte_size, Tictoctimer
import torch
from torch import nn

import numpy as np

from sklearn.preprocessing import PolynomialFeatures

from scipy.io import loadmat
import scipy.linalg as lin
import os

from utils import load_data
from data_processing import add_gaussian_noise

import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import time
from copy import deepcopy
import itertools

import re

from feature_construction import feature_library
import polynomial as p
import fourier as f

from SI_SUBNET import SS_encoder_general_eq, h_identity, e_identity, simple_Linear
from SI_metrics import plot_coeff_grid, display_equation
from run_SISUBNET import run_SISUBNET

import random
import string

from scipy.io import loadmat, savemat
import json
import pickle

c:\Users\Joost\anaconda3\envs\test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nf = 1
gamma = 1e-4
epo_idx = 25

pruning = True
mode = None
T_idx = None

pre_train = False if nf==1 else True
batch_size = 256
epochs = 150
no_reg_epochs = 50

threshold = np.array([1e-5, 2e-5, 4e-5, 8e-5, 1e-4, 2e-4, 4e-4, 8e-4, 1e-3, 2e-3, 4e-3, 8e-3, 1e-2, 2e-2, 4e-2, 8e-2, 1e-1])

In [3]:
data_set_name="sc_wnl_noisy" # data



x_data, u_data, y_data, eq, T, U, set_idx = load_data(set=data_set_name)
[N_arrow, N_train, N_val, N_test] = set_idx[0]

if T is None:
    T = np.eye(x_data.shape[-1])
if U is None:
    U = np.eye(u_data.shape[-1])

# data organizing
x_arrow, u_arrow = x_data[:N_arrow], u_data[:N_arrow,0]
x_train, u_train = x_data[N_arrow:N_arrow+N_train-N_val,:], u_data[N_arrow:N_arrow+N_train-N_val,0]
x_val, u_val     = x_data[N_arrow+N_train-N_val:-N_test,:], u_data[N_arrow+N_train-N_val:-N_test,0]
x_test, u_test   = x_data[-N_test:,:], u_data[-N_test:]

# system data
arrow   = System_data(u=u_arrow, y=x_arrow)
train   = System_data(u=u_train, y=x_train)
val     = System_data(u=u_val[:1000], y=x_val[:1000,:])
test    = System_data(u=u_test, y=x_test)

# function lib
functions = [p.f, p.f2, p.f3, p.f4, p.f5, p.f6]
poly = feature_library(functions=functions, nx=2, nu=1, T=T, U=U)

# network param
nx, nu = 2, 1 # state dimension and inputs
na, nb = 0, 0
na_right = 1

f_net_kwargs = {"feature_library": poly, "u": nu, "nf": poly.feature_number()}

# output model
h_net = h_identity
h_net_kwargs = {}

# encoder model
e_net = e_identity
e_net_kwargs = {}

Data loaded from sc_wnl_noisy!


In [4]:
def convert_float_to_string(number):
    coefficient, exponent = f"{number:.1e}".split('e')
    return f"{int(float(coefficient))}em{-int(exponent)}"

In [5]:
for th in threshold:
    set_name = "nf10_th_wnl_poly_{}".format(convert_float_to_string(th))
    print(set_name)

    run_SISUBNET(nx, na, nb, na_right, \
             f_net_kwargs, e_net, e_net_kwargs, h_net, h_net_kwargs,\
             arrow, train, val, test, data_set_name,\
             nf, gamma, mode, T_idx, pruning, th, epo_idx,\
             batch_size, pre_train=pre_train, no_reg_epochs=no_reg_epochs, nf_epochs=epochs,\
             save_data=True, set_name=set_name, verbose=True)

nf10_th_wnl_poly_1em1
start
Initilizing the model and optimizer
Size of the training array =  11.0 MB
N_training_samples = 179999, batch_size = 256, N_batch_updates_per_epoch = 703
Initial Validation sim-NRMS= 1.0396485637659576


  1%|          | 1/100 [00:01<03:01,  1.83s/it]

########## New lowest validation loss achieved ########### sim-NRMS = 0.618724144293747
Epoch    1, sqrt loss 0.06509, Val sim-NRMS 0.6187, Time Loss: 73.0%, data: 4.3%, val: 22.0%, 383.8 batches/sec


  2%|▏         | 2/100 [00:03<02:52,  1.76s/it]

########## New lowest validation loss achieved ########### sim-NRMS = 0.3424620576225546
Epoch    2, sqrt loss 0.02463, Val sim-NRMS 0.3425, Time Loss: 72.4%, data: 4.6%, val: 22.4%, 397.3 batches/sec


...

 21%|██▏       | 32/150 [06:01<22:42, 11.54s/it]

Epoch   32, sqrt loss  0.1329, Val sim-NRMS  1.004, Time Loss: 94.2%, data: 0.8%, val: 4.7%, 62.3 batches/sec


 21%|██▏       | 32/150 [06:03<22:20, 11.36s/it]


Stopping early due to a KeyboardInterrupt
Model already initilized (init_model_done=True), skipping initilizing of the model, the norm and the creation of the optimizer
Size of the training array =  48.1 MB
N_training_samples = 179990, batch_size = 256, N_batch_updates_per_epoch = 703
Initial Validation sim-NRMS= 1.004473111786454


  0%|          | 0/50 [00:00<?, ?it/s]